In [2]:
import pandas as pd
import utils.hebrew_tokenizer as ht

# df = pd.read_csv('./datasets/hewiki/hebrew_nikud_dataset_filtered.csv')
df = pd.read_csv('./datasets/hewiki/hebrew_nikud_dataset_with_uncertainty.csv')

In [3]:
df.head()

,text,nikud_mask,article_title,article_length,text_length,nikud_mask_length,uncertainty_mask
0,"תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,141,141,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,44,45,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,"חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,85,86,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"תורת האֶתֶר גרסה שיש חומר הממלא את כל המרחב, ו...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,70,70,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,בניסוי מייקלסון-מורלי נמדדה מהירות הקפת כדור ה...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,151,151,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [4]:
import ast

def mask_str(mask):
    if isinstance(mask, str):
        mask = ast.literal_eval(mask)
    return "".join([str(x) for x in mask])

In [4]:
from tqdm.notebook import tqdm

mask_lengths = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Checking mask lengths"):
    mask_lengths.append(len(ht.get_nikud_mask(row['text'])[1]))

mismatch_rows = df[[mask_len != row['nikud_mask_length'] for mask_len, (_, row) in zip(mask_lengths, df.iterrows())]]
mismatch_rows.shape

Checking mask lengths:   0%|          | 0/81995 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2611 > 2048). Running this sequence through the model will result in indexing errors


(788, 7)

In [5]:
mismatch_rows['diff'] = [len(ht.get_nikud_mask(row['text'])[1]) - row['nikud_mask_length'] for _, row in mismatch_rows.iterrows()]
grouped = mismatch_rows.groupby('diff')
grouped.size()

C:\Users\eitam\AppData\Local\Temp\ipykernel_3812\2918813076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatch_rows['diff'] = [len(ht.get_nikud_mask(row['text'])[1]) - row['nikud_mask_length'] for _, row in mismatch_rows.iterrows()]


diff
-3      1
-2     59
-1    728
dtype: int64

In [6]:
x = grouped.get_group(-1).iloc[10]
print(x['text'])
input_ids, nikud_mask = ht.get_nikud_mask(x['text'])
print(ht.tokenizer.decode(input_ids))
print(mask_str(nikud_mask))
print(mask_str(x['nikud_mask']))

גַ'לְג'וּלְיָה (בערבית: جلجولية) היא מועצה מקומית ערבית-מוסלמית במחוז המרכז בישראל, 3 ק"מ מזרחית לכפר סבא
[CLS] ג'ל ג'ו ל י ה   ( ב ע ר ב י ת :   [UNK] )   ה י א   מ ו ע צ ה   מ ק ו מ י ת   ע ר ב י ת - מ ו ס ל מ י ת   ב מ ח ו ז   ה מ ר כ ז   ב י ש ר א ל,   3   ק " מ   מ ז ר ח י ת   ל כ פ ר   ס ב א [SEP]
010100111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0010100111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [7]:
sample = df.sample(1)
sample.iloc[0]

text                 פרנטישק לַאנגֶר (בצ'כית: František Langer; 3 ב...
nikud_mask           [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ...
article_title                                            פרנטישק לאנגר
article_length                                                     537
text_length                                                        134
nikud_mask_length                                                  134
uncertainty_mask     [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, ...
Name: 45706, dtype: object

In [9]:
input_ids, nikud_mask = ht.get_nikud_mask(sample.iloc[0]['text'])
print(ht.tokenizer.decode(input_ids), len(mask_str(nikud_mask)), len(mask_str(sample.iloc[0]['nikud_mask'])))

[CLS] נ ו ל ד   ב ר ו ב ה   ( r o u b a i x )   ש ע ל   ג ב ו ל   ב ל ג י ה   ל מ ש פ ח ה   מ ו ז י ק ל י ת [SEP] 53 53


In [10]:

uncertainty_mask, ambiguous_chars = ht.get_nikud_uncertainty(sample.iloc[0]['text'])
sample.iloc[0]['text'], mask_str(sample.iloc[0]['nikud_mask']), mask_str(uncertainty_mask), ambiguous_chars

('נולד ברוּבֶּה (Roubaix) שעל גבול בלגיה למשפחה מוזיקלית',
 '00000000110000000000000000000000000000000000000000000',
 '00000000110000000000000000000000000000000000000000000',
 [{'char': 'ו', 'position': (tensor(7), tensor(8))},
  {'char': 'ב', 'position': (tensor(9), tensor(10))}])

In [75]:
small_df = df.sample(1000)

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas(desc="Calculating uncertainty masks")
small_df['uncertainty_mask'] = small_df['text'].progress_apply(lambda text: ht.get_nikud_uncertainty(text)[0])


Calculating uncertainty masks:   0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
def column_filter(row):
    text = row['text']
    mask = ast.literal_eval(row['nikud_mask'])
    return ht.has_1s_run(text, mask, min_run=1, max_run=3)

filtered_df = df[df.apply(column_filter, axis=1)]

Token indices sequence length is longer than the specified maximum sequence length for this model (2609 > 2048). Running this sequence through the model will result in indexing errors


In [7]:
filtered_df.head()

,text,nikud_mask,article_title,article_length,text_length,nikud_mask_length,uncertainty_mask
0,"תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,141,141,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,44,45,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,"חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,85,86,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"תורת האֶתֶר גרסה שיש חומר הממלא את כל המרחב, ו...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,70,70,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,בניסוי מייקלסון-מורלי נמדדה מהירות הקפת כדור ה...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,151,151,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [8]:
from tqdm.notebook import tqdm

tqdm.pandas()  # Enable tqdm for pandas

filtered_df['nikud_word_mask'] = filtered_df.progress_apply(
    lambda row: ht.convert_token_to_word_mask(row['text'], row['nikud_mask']), axis=1)
filtered_df['uncertainty_word_mask'] = filtered_df.progress_apply(
    lambda row: ht.convert_token_to_word_mask(row['text'], row['uncertainty_mask']), axis=1)

  0%|          | 0/65533 [00:00<?, ?it/s]

C:\Users\eitam\AppData\Local\Temp\ipykernel_2112\2175315501.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['nikud_word_mask'] = filtered_df.progress_apply(


  0%|          | 0/65533 [00:00<?, ?it/s]

C:\Users\eitam\AppData\Local\Temp\ipykernel_2112\2175315501.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['uncertainty_word_mask'] = filtered_df.progress_apply(


In [9]:
filtered_df.to_csv('./datasets/hewiki/hebrew_nikud_dataset_filtered_with_word_masks.csv', index=False)

In [13]:
len(filtered_df.iloc[0]['uncertainty_word_mask']), len(filtered_df.iloc[0]['nikud_word_mask'])
# convert_token_to_word_mask(filtered_df.iloc[0]['text'], filtered_df.iloc[0]['uncertainty_mask'])
# convert_token_to_word_mask(filtered_df.iloc[0]['text'], filtered_df.iloc[0]['uncertainty_mask'])

(23, 23)

In [ ]:

# Compute the overlap ratio in a separate step
def overlap_ratio(row):
    uncertainty_mask = row['uncertainty_word_mask']
    nikud_mask = row['nikud_word_mask']
    if isinstance(nikud_mask, str):
        nikud_mask = ast.literal_eval(nikud_mask)
    overlap = sum(1 for u, n in zip(uncertainty_mask, nikud_mask) if u == 1 and n == 1)
    total_nikud = sum(nikud_mask)
    return overlap / total_nikud if total_nikud > 0 else None

def opposite_overlap_ratio(row):
    uncertainty_mask = row['uncertainty_word_mask']
    if isinstance(uncertainty_mask, str):
        uncertainty_mask = ast.literal_eval(uncertainty_mask)
    nikud_mask = row['nikud_word_mask']
    if isinstance(nikud_mask, str):
        nikud_mask = ast.literal_eval(nikud_mask)
    overlap = sum(1 for u, n in zip(uncertainty_mask, nikud_mask) if u == 1 and n == 1)
    total_uncertainty = sum(uncertainty_mask)
    return overlap / total_uncertainty if total_uncertainty > 0 else None

C:\Users\eitam\AppData\Local\Temp\ipykernel_3812\3302538117.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['uncertainty_overlap_ratio'] = filtered_df.apply(overlap_ratio, axis=1)


count    61301.000000
mean         0.635086
std          0.468374
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: uncertainty_overlap_ratio, dtype: float64

In [ ]:
filtered_df['uncertainty_overlap_ratio'] = filtered_df.apply(overlap_ratio, axis=1)
filtered_df['uncertainty_overlap_ratio'].describe()

In [ ]:
filtered_df['uncertainty_overlap_ratio_2'] = filtered_df.apply(opposite_overlap_ratio, axis=1)
filtered_df['uncertainty_overlap_ratio_2'].describe()

In [15]:
not_enough_overlap = filtered_df[filtered_df['uncertainty_overlap_ratio'] <= 0.5]
test = not_enough_overlap.sample(5)

In [17]:
for i in range(len(test)):
    print(f"Example {i+1}:")
    print("Text:", test.iloc[i]['text'])
    print("Nikud Mask:      ", mask_str(test.iloc[i]['nikud_word_mask']))
    print("Uncertainty Mask:", mask_str(test.iloc[i]['uncertainty_word_mask']))
    print("Overlap Ratio:", test.iloc[i]['uncertainty_overlap_ratio'])
    print()

Example 1:
Text: ברבדוס נחשבה לאומה הבריטית ביותר בקֶרֶב חבר העמים הקריבי הבריטי, והיוותה את התממשות השליטה הבלתי מעורערת שיושמה על ידי הבריטים משנת 1625 ועד למתן העצמאות בשנת 1966
Nikud Mask:       000001000000000000000000000
Uncertainty Mask: 100000001000000000000000000
Overlap Ratio: 0.0

Example 2:
Text: 6 הקלטות הופצו בסדרת המשך זו, וכל אחת הכילה שלושה ספרים מ"סדרת הרַכבות"
Nikud Mask:       0000000000001
Uncertainty Mask: 0000000000000
Overlap Ratio: 0.0

Example 3:
Text: משך תקופה פעל כמעצב בצפון איטליה בשיתוף פעולה עם חברה (CALOIׂ) שצברה ניסיון ומוניטין בריהוט לבתי אופרה גדולים באירופה
Nikud Mask:       0000000000100000000
Uncertainty Mask: 0000000000000000000
Overlap Ratio: 0.0

Example 4:
Text: ישראל פכט נולד בווינה, בנם של יוכבד אֶווה (לבית פרמינגר) ובנימין יצחק (בֶּנו) פכט
Nikud Mask:       00000001000010
Uncertainty Mask: 01000001010001
Overlap Ratio: 0.5

Example 5:
Text: תומנישווילי-צרתלי פרסמה קבצים ספרותיים, שהיוו אוספים של סיפוריה וסיפורים מתורגמים או מעובדים משפות זר

In [80]:
_, ambiguous = ht.get_nikud_uncertainty(not_enough_overlap.iloc[0]['text'])
ambiguous

[{'char': 'ק',
  'position': (tensor(5), tensor(6)),
  'entropy': 1.7002260453349685,
  'margin': 0.10639014840126038,
  'max_prob': 0.46183285117149353,
  'top_candidates': [('ְּ', 0.46183285117149353),
   ('ָּ', 0.35544270277023315),
   ('ַּ', 0.14430095255374908),
   ('ִּ', 0.025373805314302444),
   ('', 0.004338645841926336)]},
 {'char': 'מ',
  'position': (tensor(6), tensor(7)),
  'entropy': 1.8923375398731663,
  'margin': 0.3622446358203888,
  'max_prob': 0.5610603094100952,
  'top_candidates': [('ָ', 0.5610603094100952),
   ('ְ', 0.19881567358970642),
   ('ָּ', 0.1026516854763031),
   ('ֵ', 0.0913892388343811),
   ('ַ', 0.023114318028092384)]},
 {'char': 'ל',
  'position': (tensor(28), tensor(29)),
  'entropy': 1.007569559218598,
  'margin': 0.2989606559276581,
  'max_prob': 0.6457830667495728,
  'top_candidates': [('ַ', 0.6457830667495728),
   ('ָ', 0.3468224108219147),
   ('ְ', 0.0030318722128868103),
   ('', 0.0022456843871623278),
   ('ַּ', 0.0006822854629717767)]},
 {'char'